# Анализ Траектории 

Запомнить содержимое папки:

In [1]:
import subprocess
restart = subprocess.check_output(['ls']).decode('utf-8').strip()
with open('restart.txt', 'w') as file:
    file.write(restart)
    file.write("restart.txt")
print("Изначальное состояние сохранено")

Изначальное состояние сохранено


Удалить все лишнее и начать заново 

In [2]:
import os
import shutil

with open('restart.txt', 'r') as file:
    restart = file.read().strip()

for item in os.listdir():
    if item != 'restart.txt' and item not in restart:
        if os.path.isfile(item):
            os.remove(item)
        elif os.path.isdir(item):
            shutil.rmtree(item)

print("Изначальное состояние восстановлено")

for file_name in os.listdir():
    if file_name.startswith('#') and file_name.endswith('#'):
        os.remove(file_name)

Изначальное состояние восстановлено


## Параметры

In [3]:
import os
current_dir = os.path.basename(os.getcwd())
ligand = ''.join(filter(str.isdigit, current_dir))

protein='7uj7'                              # Замените на реальное имя белка 
ligand_group_number = 13                    # Обычно номер группы лиганда это 13
trajectory = f'ER-{ligand}_MD_fit.xtc'      # Заменить на вашу траекторию
structure = f'ER-{ligand}_MD0.gro'          # Заменить на вашу структуру
output = 'output.xvg'                       # Заменить на ваш вывод
num_tread=16                                # Количество ядер ПК
name=f"ER-{ligand}_MD0"                     # Замените на имя вашей структуры
LBP = "r343|r346|r349|r350|r353|r383|r384|r387|r388|r391|r394|r404|r421|r424|r425|r428|r524|r525"

## Подготовка 

In [4]:
import os
import subprocess

def run_gmx_command(commands, group_name, structure_file, ndx_file):
    if not os.path.isfile(structure_file):
        print(f'Ошибка: Файл {structure_file} не существует')
        return
    
    if not os.path.isfile(ndx_file):
        print(f'Ошибка: Файл {ndx_file} не существует')
        return

    result = subprocess.run(f'echo "{commands}" | gmx make_ndx -f {structure_file} -n {ndx_file} -o {ndx_file}',
                            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='utf-8')

    if result.returncode == 0:
        print(f'Группа "{group_name}" успешно создана')
    else:
        print('Произошла ошибка при выполнении команды')
        print('Стандартный вывод:', result.stdout)
        print('Стандартный вывод ошибок:', result.stderr)

## 2. Вычисляем RMSD  
Вычисляем RMSD для основной цепи белка __(backbone)__, лиганда без водородов, сайты связывания без водородов:

In [5]:
import os
import re
import shutil
import subprocess
import pandas as pd
import matplotlib.pyplot as plt

def extract_labels(file_name):
    title = xlabel = ylabel = None
    with open(file_name, "r") as file:
        for line in file:
            if re.match(r"@ *title", line):
                title = re.sub(r"@ *title *\"", "", line).strip().strip("\"")
            elif re.match(r"@ *xaxis", line):
                xlabel = re.sub(r"@ *xaxis *label *\"", "", line).strip().strip("\"")
            elif re.match(r"@ *yaxis", line):
                ylabel = re.sub(r"@ *yaxis *label *\"", "", line).strip().strip("\"")
    return title, xlabel, ylabel

def run_subprocess_command(commands, group, ligand, trajectory, structure):
    file_name = f'RMSD/{group}/{ligand}_{group}_rmsd.xvg'
    result = subprocess.run(f'echo "{commands}" | gmx rms -f {trajectory} -s {structure} -n DRY.ndx -tu ns -o {file_name}', 
                            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='utf-8')
    return result, file_name

def clean_directory(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

def plot_data(data, xlabel, ylabel, title):
    lower = data[ylabel].quantile(0.0001)
    upper = data[ylabel].quantile(0.9999)
    data[ylabel] = data[ylabel].rolling(window=2).mean()
    plt.figure(figsize=(12, 6))
    plt.plot(data[xlabel], data[ylabel], marker="o", markersize=1, linestyle="-", linewidth=0.5)
    plt.ylim(lower, upper)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

def process_group(group, commands, ligand, trajectory, structure):
    clean_directory(f'RMSD/{group}')
    result, file_name = run_subprocess_command(commands, group, ligand, trajectory, structure)
    if result.returncode == 0:
        print(f'Команда успешно выполнена для группы "{group}"')
        title, xlabel, ylabel = extract_labels(file_name)
        data = pd.read_csv(file_name, skiprows=17, delimiter="\s+", header=None, names=[xlabel, ylabel], comment="@")
        plot_data(data, xlabel, ylabel, title)
    else:
        print(f'Произошла ошибка при выполнении команды для группы "{group}"')
        print('Стандартный вывод ошибок:', result.stderr)

clean_directory('RMSD')
group_commands = {'backbone': '4\n4', f'{ligand}_NOH': '4\n4', 'LBP': '16\n16'}
for group, commands in group_commands.items():
    process_group(group, commands, ligand, trajectory, structure)

Произошла ошибка при выполнении команды для группы "backbone"
Стандартный вывод: 
Стандартный вывод ошибок:                  :-) GROMACS - gmx rms, 2022.4-conda_forge (-:

Executable:   /home/sergei/App/Anaconda3/envs/gmxMMPBSA/bin.AVX2_256/gmx
Data prefix:  /home/sergei/App/Anaconda3/envs/gmxMMPBSA
Working dir:  /home/sergei/Documents/ERa/MD/ER-116
Command line:
  gmx rms -f ER-116_MD_fit.xtc -s ER-116_MD0.gro -n DRY.ndx -tu ns -o RMSD/backbone/116_backbone_rmsd.xvg


-------------------------------------------------------
Program:     gmx rms, version 2022.4-conda_forge
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int*, char**)

Error in user input:
Invalid command-line options
  In command-line option -s
    File 'ER-116_MD0.gro' does not exist or is not accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 2)

For more information and tips 

Лиганд выравниванием на __backbone__!!! В остальных случаях выравниваем группы сами на себя.

## 3. Делаем кластеризацию

Делаем кластеризацию для backbone, лиганда (без Н) и сайта связывания (без Н). О там как это делать [читаем здесь](https://vk.com/@501384604-klasternyi-analiz-v-gromacs-gmx-cluster).  

__cutoff__  — отсечка в нм(!!!), его значение нужно изменять для получения оптимальной кластеризации  
__start_num__ - С какой нс  
__end_num__ - По какую нс

In [6]:
group_params = {
    'backbone': {
        'cutoff': '0.15',
        'start_num': '90',
        'end_num': '190'
    },
    f'{ligand}_NOH': {
        'cutoff': '0.15',
        'start_num': '90',
        'end_num': '190'
    },
    'LBP': {
        'cutoff': '0.15',
        'start_num': '200',
        'end_num': '300'
    }
}

### Команда для кластеризации


In [7]:
import os
import re
import shutil
import subprocess
import pandas as pd
import matplotlib.pyplot as plt

def extract_labels(file_name):
    title = xlabel = ylabel = None
    with open(file_name, "r") as file:
        for line in file:
            if re.match(r"@ *title", line):
                title = re.sub(r"@ *title *\"", "", line).strip().strip("\"")
            elif re.match(r"@ *xaxis", line):
                xlabel = re.sub(r"@ *xaxis *label *\"", "", line).strip().strip("\"")
            elif re.match(r"@ *yaxis", line):
                ylabel = re.sub(r"@ *yaxis *label *\"", "", line).strip().strip("\"")
    return title, xlabel, ylabel

def copy_files_to_cluster_dir(cluster_dir, trajectory, structure):
    shutil.copy(trajectory, cluster_dir)
    shutil.copy(structure, cluster_dir)
    shutil.copy("DRY.ndx", cluster_dir)

def run_clustering(commands, trajectory, structure, cutoff, start_num, end_num):
    return subprocess.run(
        f'echo "{commands}" |gmx cluster -f {trajectory} -s {structure} -n DRY.ndx -cutoff {cutoff} -b {start_num} -e {end_num} -tu ns -dt 0.1 -method gromos -clid {group}_md_clid.xvg -dist clusters_dist -o -g -cl',
        shell=True,            
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        encoding='utf-8'
    )

def plot_data(file_name):
    if os.path.isfile(file_name):
        title, xlabel, ylabel = extract_labels(file_name)
        data = pd.read_csv(file_name, skiprows=17, delimiter="\s+", header=None, names=[xlabel, ylabel], comment="@")
        plt.figure(figsize=(12, 6))
        if file_name.endswith("_clid.xvg"):
            plt.plot(data[xlabel], data[ylabel], marker="o", markersize=2, linestyle="", color ="black")
        else:
            plt.plot(data[xlabel], data[ylabel], marker="o", markersize=1, linestyle="-", linewidth=0.5)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.show()
    else:
        print(f"File {file_name} not found.")

orig_dir = os.getcwd()

for group in sorted(os.listdir('RMSD'), reverse=True):
    cluster_dir = os.path.join('RMSD', group)
    if not os.path.isdir(cluster_dir):
        continue
    copy_files_to_cluster_dir(cluster_dir, trajectory, structure)
    try:
        os.chdir(cluster_dir)
        params = group_params[group]
        commands = "16\n16" if group == 'LBP' else "4\n4"
        result = run_clustering(commands, trajectory, structure, params['cutoff'], params['start_num'], params['end_num'])
        if result.returncode == 0:
            print(f'Кластеризация успешно выполнена для группы "{group}"')
            for file_name in ["clusters_dist.xvg",f'{ligand}_{group}_rmsd.xvg', f"{group}_md_clid.xvg"]:
                plot_data(file_name)
        else:
            print(f'Произошла ошибка при выполнении кластеризации для группы "{group}"')
            print('Стандартный вывод:', result.stdout)
            print('Стандартный вывод ошибок:', result.stderr)
    finally:
        os.chdir(orig_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'ER-116_MD0.gro'

### Извлекаем репрезентативные структуры

In [ ]:
import nglview as nv
import os
import subprocess

def visualize_pdb(file_path):
    view = nv.show_structure_file(file_path)

    view.background = 'black'
    view.add_representation('cartoon', selection='protein')
    view.add_representation('ball+stick', selection='ligand')
    view.add_representation('surface', selection='protein', 
                            surfaceType='vws', color='grey', 
                            opacity=0.5)
    
    return view

views = []

orig_dir = os.getcwd()

for group in sorted(os.listdir('RMSD'), reverse=True):
    cluster_dir = os.path.join('RMSD', group)
    
    if not os.path.isdir(cluster_dir):
        continue

    os.chdir(cluster_dir)
    
    params = group_params[group]
    start_num = params['start_num']
    end_num = params['end_num']

    try:
        result = subprocess.run(
            f'echo "14"|gmx trjconv -f {trajectory} -s {structure} -n DRY.ndx -b 100 -e 500 -o LIG_rep_structure.pdb -dt 0.1 -tu ns -dump 170',
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            encoding='utf-8')
        
        if result.returncode == 0:
            print(f'Репрезентативная структура успешно извлечена для группы "{group}"')
            if os.path.isfile('LIG_rep_structure.pdb'):
                views.append(visualize_pdb('LIG_rep_structure.pdb'))
            else:
                print("Файл LIG_rep_structure.pdb не найден.")
        else:
            print(f'Произошла ошибка при извлечении репрезентативной структуры для группы "{group}"')
            print('Стандартный вывод:', result.stdout)
            print('Стандартный вывод ошибок:', result.stderr)
    finally:
        os.chdir(orig_dir)
        
display(views[0])

## 4. Вычисляем энергии связывания (энтальпийные) методом MMGBSA

Участок выбираем согласно графику RMSD лиганда и распределению кластеров.

In [ ]:
start_frame=100
end_frame=150

In [ ]:
import os
import shutil
import subprocess

def copy_files(src_files, dest_dir):
    for file in src_files:
        try:
            shutil.copy(file, dest_dir)
        except IOError as e:
            print(f"Не удалось скопировать файл {file} Ошибка: {e}")
        except:
            print(f"Неожиданная ошибка при копировании файла {file}")

def run_mmpbsa(start_frame, end_frame, ligand_group_number, ligand, trajectory):
    mmpbsa_in_content = f"""
    &general
    forcefields="oldff/leaprc.ff99SBildn",leaprc.gaff"
    startframe={start_frame}00, endframe={end_frame}00, interval=10,
    /
    &gb
    intdiel=4,
    extdiel=80.0
    igb=5, saltcon=0.150,
    /
    &decomp
    idecomp=2, dec_verbose=1,
    print_res="within 8"
    /
    """
    with open('mmpbsa.in', 'w') as file:
        file.write(mmpbsa_in_content);

    with open(os.devnull, 'w') as devnull:
        subprocess.call(['obabel', 'complex.gro', '-O', 'complex.pdb'], stdout=devnull, stderr=devnull)

    print("Расчет энергий связывания начат")
    command = f'mpirun -np 16 gmx_MMPBSA -O -i mmpbsa.in -cs complex.pdb -ci DRY.ndx -cg 1 {ligand_group_number} -ct {trajectory} -lm {ligand}.mol2 -cp topol.top -o ER-{ligand}_MMPBSA.dat -eo ER-{ligand}_MMPBSA.csv -do ER-{ligand}_DECOMP_MMPBSA.dat -deo ER-{ligand}_DECOMP_MMPBSA.csv'
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print("Расчет энергий связывания завершен")

shutil.rmtree('gmx_MMPBSA', ignore_errors=True)
os.makedirs('gmx_MMPBSA', exist_ok=True)

files_to_copy = ['complex.gro', 'DRY.ndx', trajectory, f'{ligand}.mol2', 'topol.top', 'atoms.itp', f'{ligand}.itp', f'{protein}.itp']
copy_files(files_to_copy, 'gmx_MMPBSA')

current_dir = os.getcwd()
try:
    os.chdir('gmx_MMPBSA')
    run_mmpbsa(start_frame, end_frame, ligand_group_number, ligand, trajectory)
finally:
    os.chdir(current_dir)
    

## 5. Анализ взаимодействий

### Подготовка траекторий

In [ ]:
import os
import shutil
import subprocess
import MDAnalysis as mda
import prolif as plf
from prolif.plotting.network import LigNetwork
import glob

def modify_line(line):
    parts = line.split('=')
    if len(parts) > 1:
        values = parts[1].split()
        if len(values) > 1:
            return parts[0] + '= ' + values[0] + ' ' + ' '.join(values[2:]) + '\n'
    return line

with open('nvt.mdp', 'r') as file:
    lines = file.readlines()

with open('test.mdp', 'w') as file:
    for line in lines:
        if line.startswith('tau_t') or line.startswith('ref_t'):
            line = modify_line(line)
        elif 'Water_and_ions' in line:
            line = line.replace('Water_and_ions', '                ')
        file.write(line)

with open('topol.top', 'r') as file:
    lines = file.readlines()

with open('top.top', 'w') as file:
    for line in lines:
        if 'SOL' not in line and 'NA' not in line and 'CL' not in line:
            file.write(line)

subprocess.run(f'gmx grompp -f test.mdp -c {structure} -r {structure} -p top.top -n DRY.ndx -o test.tpr -maxwarn 1',
               shell=True,
               stdout=subprocess.PIPE,
               stderr=subprocess.PIPE,
               encoding='utf-8');


folder_name = 'Prolif'
shutil.rmtree(folder_name, ignore_errors=True)
os.makedirs(folder_name, exist_ok=True)

files_to_copy = [f'ER-{ligand}_MD_fit.xtc', 'test.tpr']

for file in files_to_copy:
    shutil.copy(file, folder_name)

step = 360 # Chose a step in degrees to rotate the picture "

os.chdir(folder_name)

traj = glob.glob('./*.xtc')[0][2:]
top = glob.glob('./*.tpr')[0][2:]
NAME = traj[0:-11]

u = mda.Universe(top, traj)
prot = u.select_atoms("protein")
lig = u.select_atoms(f"resname {ligand}")
lmol = plf.Molecule.from_mda(lig)
pmol = plf.Molecule.from_mda(prot)

fp_l = plf.Fingerprint()
fp_l.run(u.trajectory[start_frame:end_frame:step], lig, prot)
df_l = fp_l.to_dataframe(return_atoms=True)
df = fp_l.to_dataframe(return_atoms=False)
D = df.groupby(level=["ligand", "protein"], axis=1, sort=False).sum().astype(bool).mean()
un_lig = D.index.get_level_values("protein").unique().tolist()
un_lig.sort()
lst1 = []
for el in un_lig:
    lst1.append(el[0:3] + str(int(el[3:]) + 306))

mapping = dict(zip(un_lig, lst1))

df_l = df_l.rename(mapping, axis='columns')
D = df_l

for j in range(0, 360, step):
    net_l = LigNetwork.from_ifp(df_l, lmol, kind="aggregate", threshold=0.5, rotation=j)
    net_l.save(f'{NAME}_ints_{j}.html')

df_n2 = fp_l.to_dataframe(return_atoms=False)
df_n2 = df_n2.rename(mapping, axis='columns')

occ2 = df_n2.mean()
E = occ2.loc[occ2 > 0.5]
E = E.round(decimals=1)
E.to_csv(NAME + '_ints.csv', sep=',')